In [ ]:
import whisper
import subprocess
from pathlib import Path
model = whisper.load_model("base")

In [ ]:
# model = whisper.load_model("turbo")

In [ ]:
def toWav16k(inPath: Path, outPath: Path) -> Path:
    """
    Convierte cualquier entrada (audio o video) a WAV PCM 16 kHz mono.
    -vn fuerza a ignorar video; aresample=soxr mejora la calidad del remuestreo.
    """
    cmd = [
        "ffmpeg", "-y", "-i", str(inPath),
        "-vn",
        "-ac", "1",
        "-af", "aresample=resampler=soxr:precision=33",
        "-ar", "16000",
        "-c:a", "pcm_s16le",
        str(outPath)
    ]
    subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return outPath

def prepareAudioForWhisper(srcPath: Path, enabled: bool = True) -> Path:
    """
    Si enabled=True → genera 'input.16k.wav' y lo devuelve.
    Si enabled=False → devuelve el path original sin tocar.
    """
    if not enabled:
        return srcPath
    target = srcPath.parent / "input.16k.wav"
    return toWav16k(srcPath, target)

# --- Uso sencillo ---
model = whisper.load_model("base")

audioPath = Path("./audio.mp3")
normalizedPath = prepareAudioForWhisper(audioPath, enabled=True)

result = model.transcribe(str(normalizedPath))
print(result["text"])

In [ ]:
result["text"]